In [2]:
import pandas as pd

# Load the uploaded files
dataset_path = 'dataset.xlsx'
preprocessed_data_path = 'preprocessed_data.csv'

# Read the dataset files
dataset = pd.read_excel(dataset_path)
preprocessed_data = pd.read_csv(preprocessed_data_path)

# Display the first few rows of both files to understand their structure
dataset_head = dataset.head()
preprocessed_data_head = preprocessed_data.head()

dataset_head, preprocessed_data_head


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5756\3742617597.py:9: DtypeWarning: Columns (2,3,4,5,6,7,9,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  preprocessed_data = pd.read_csv(preprocessed_data_path)


(   Unnamed: 0  Unnamed: 1  angk status_akhir   rencana_berangkat  \
 0         NaN         NaN  2019        Lulus 2019-11-29 18:30:00   
 1         NaN         NaN  2019        Lulus 2019-12-13 17:00:00   
 2         NaN         NaN  2019        Lulus 2019-12-19 08:00:00   
 3         NaN         NaN  2019        Lulus 2020-01-15 13:00:00   
 4         NaN         NaN  2019        Lulus 2020-02-14 17:00:00   
 
       rencana_kembali                      desc  \
 0 2019-12-01 21:00:00  Ingin Bertemu Orang Tua    
 1 2019-12-15 21:00:00  Ingin bertemu orang tua    
 2 2020-01-05 18:00:00        Libur akhir tahun    
 3 2020-01-26 19:00:00            libur semester   
 4 2020-02-16 21:00:00         Bertemu Orang Tua   
 
                              tujuan  realisasi_berangkat    realisasi_kembali  \
 0              medan,Sumatera Utara  2019-11-29 18:01:30  2019-12-01 20:00:52   
 1  Doloksanggul, Humbang Hasundutan  2023-02-13 21:51:53  2023-02-13 21:51:58   
 2       Doloksanggul,su

In [3]:
# Preprocessing steps for dataset

# Convert date columns to datetime format for processing
date_columns = ['rencana_berangkat', 'rencana_kembali', 'realisasi_berangkat', 'realisasi_kembali']
for col in date_columns:
    dataset[col] = pd.to_datetime(dataset[col], errors='coerce')

# Calculate additional features:
# 1. 'durasi' - duration of trip in hours
dataset['durasi'] = (dataset['rencana_kembali'] - dataset['rencana_berangkat']).dt.total_seconds() / 3600

# 2. 'hari_berangkat' and 'hari_kembali' - day of the week for departure and return
dataset['hari_berangkat'] = dataset['rencana_berangkat'].dt.day_name()
dataset['hari_kembali'] = dataset['rencana_kembali'].dt.day_name()

# 3. 'range_waktu' - categorize the time of day for departure
def categorize_time_of_day(time):
    if time.hour < 12:
        return 'Pagi'
    elif 12 <= time.hour < 16:
        return 'Siang'
    elif 16 <= time.hour < 20:
        return 'Sore'
    else:
        return 'Malam'

dataset['range_waktu'] = dataset['rencana_berangkat'].dt.time.apply(lambda x: categorize_time_of_day(x) if pd.notnull(x) else None)

# Assign 'izin' category based on example data (mocked here as 'DATA IB' for all rows)
dataset['izin'] = 'DATA IB'

# Preview the processed dataset
processed_dataset = dataset[
    ['angk', 'status_akhir', 'rencana_berangkat', 'rencana_kembali', 'desc',
     'tujuan', 'durasi', 'hari_berangkat', 'hari_kembali', 'range_waktu', 'izin']
]

processed_dataset.head()


,angk,status_akhir,rencana_berangkat,rencana_kembali,desc,tujuan,durasi,hari_berangkat,hari_kembali,range_waktu,izin
0,2019,Lulus,2019-11-29 18:30:00,2019-12-01 21:00:00,Ingin Bertemu Orang Tua,"medan,Sumatera Utara",50.5,Friday,Sunday,Sore,DATA IB
1,2019,Lulus,2019-12-13 17:00:00,2019-12-15 21:00:00,Ingin bertemu orang tua,"Doloksanggul, Humbang Hasundutan",52.0,Friday,Sunday,Sore,DATA IB
2,2019,Lulus,2019-12-19 08:00:00,2020-01-05 18:00:00,Libur akhir tahun,"Doloksanggul,sumatera utara",418.0,Thursday,Sunday,Pagi,DATA IB
3,2019,Lulus,2020-01-15 13:00:00,2020-01-26 19:00:00,libur semester,"Doloksanggul, Humbang Hasundutan",270.0,Wednesday,Sunday,Siang,DATA IB
4,2019,Lulus,2020-02-14 17:00:00,2020-02-16 21:00:00,Bertemu Orang Tua,"Doloksanggul, Humbang Hasundutan",52.0,Friday,Sunday,Sore,DATA IB


In [4]:
# Filter data for IK category (mocking 'IK' as part of izin for now)
ik_data = processed_dataset[processed_dataset['izin'] == 'DATA IB'].copy()  # Adjust for real IK filter if available

# Define mapping for business opportunities based on keywords in 'desc'
business_opportunities = {
    "require engineering": "Sewa bus",
    "membeli peralatan": "Toko peralatan",
    "orang tua": "Akomodasi pengunjung",
    "libur": "Travel agency",
}

# Function to categorize business opportunities
def map_business_opportunity(description):
    description = str(description).lower()
    for keyword, business in business_opportunities.items():
        if keyword in description:
            return business
    return "Lainnya"

# Apply the mapping to the IK data
ik_data['peluang_bisnis'] = ik_data['desc'].apply(map_business_opportunity)

# Count the number of students per business opportunity
business_counts = ik_data['peluang_bisnis'].value_counts()

ik_data.head(), business_counts


(   angk status_akhir   rencana_berangkat     rencana_kembali  \
 0  2019        Lulus 2019-11-29 18:30:00 2019-12-01 21:00:00   
 1  2019        Lulus 2019-12-13 17:00:00 2019-12-15 21:00:00   
 2  2019        Lulus 2019-12-19 08:00:00 2020-01-05 18:00:00   
 3  2019        Lulus 2020-01-15 13:00:00 2020-01-26 19:00:00   
 4  2019        Lulus 2020-02-14 17:00:00 2020-02-16 21:00:00   
 
                        desc                            tujuan  durasi  \
 0  Ingin Bertemu Orang Tua               medan,Sumatera Utara    50.5   
 1  Ingin bertemu orang tua   Doloksanggul, Humbang Hasundutan    52.0   
 2        Libur akhir tahun        Doloksanggul,sumatera utara   418.0   
 3            libur semester  Doloksanggul, Humbang Hasundutan   270.0   
 4         Bertemu Orang Tua  Doloksanggul, Humbang Hasundutan    52.0   
 
   hari_berangkat hari_kembali range_waktu     izin        peluang_bisnis  
 0         Friday       Sunday        Sore  DATA IB  Akomodasi pengunjung  
 1        

In [5]:
# Re-loading the dataset to ensure no errors in processing
dataset = pd.read_excel(dataset_path)

# Repeat preprocessing for IK-specific data
for col in date_columns:
    dataset[col] = pd.to_datetime(dataset[col], errors='coerce')

# Recalculate columns (durasi, hari, range waktu, etc.)
dataset['durasi'] = (dataset['rencana_kembali'] - dataset['rencana_berangkat']).dt.total_seconds() / 3600
dataset['hari_berangkat'] = dataset['rencana_berangkat'].dt.day_name()
dataset['hari_kembali'] = dataset['rencana_kembali'].dt.day_name()
dataset['range_waktu'] = dataset['rencana_berangkat'].dt.time.apply(lambda x: categorize_time_of_day(x) if pd.notnull(x) else None)

# Reapply filter and mapping for IK data
ik_data = dataset.copy()  # Assuming all data is for 'IK'; adjust if specific filter needed
ik_data['peluang_bisnis'] = ik_data['desc'].apply(map_business_opportunity)

# Recalculate business counts
business_counts = ik_data['peluang_bisnis'].value_counts()

ik_data.head(), business_counts


(   Unnamed: 0  Unnamed: 1  angk status_akhir   rencana_berangkat  \
 0         NaN         NaN  2019        Lulus 2019-11-29 18:30:00   
 1         NaN         NaN  2019        Lulus 2019-12-13 17:00:00   
 2         NaN         NaN  2019        Lulus 2019-12-19 08:00:00   
 3         NaN         NaN  2019        Lulus 2020-01-15 13:00:00   
 4         NaN         NaN  2019        Lulus 2020-02-14 17:00:00   
 
       rencana_kembali                      desc  \
 0 2019-12-01 21:00:00  Ingin Bertemu Orang Tua    
 1 2019-12-15 21:00:00  Ingin bertemu orang tua    
 2 2020-01-05 18:00:00        Libur akhir tahun    
 3 2020-01-26 19:00:00            libur semester   
 4 2020-02-16 21:00:00         Bertemu Orang Tua   
 
                              tujuan realisasi_berangkat   realisasi_kembali  \
 0              medan,Sumatera Utara 2019-11-29 18:01:30 2019-12-01 20:00:52   
 1  Doloksanggul, Humbang Hasundutan 2023-02-13 21:51:53 2023-02-13 21:51:58   
 2       Doloksanggul,sumatera

In [7]:
# Load the preprocessed data
preprocessed_data = pd.read_csv(preprocessed_data_path)

# Step 1: Filter data for IK
ik_data = preprocessed_data[preprocessed_data['izin'] == 'IK'].copy()

# Step 2: Define mapping for business opportunities based on keywords in 'desc'
business_opportunities = {
    "require engineering": "Sewa bus",
    "membeli peralatan": "Toko peralatan sehari-hari",
    "libur": "Travel agency",
    "rekreasi": "Travel agency",
    "orang tua": "Akomodasi pengunjung",
}

# Function to categorize business opportunities
def map_business_opportunity(description):
    description = str(description).lower()
    for keyword, business in business_opportunities.items():
        if keyword in description:
            return business
    return "Lainnya"

# Step 3: Apply the mapping to the IK data
ik_data['peluang_bisnis'] = ik_data['desc'].apply(map_business_opportunity)

# Count the number of students per business opportunity
business_counts = ik_data['peluang_bisnis'].value_counts()

ik_data.head(), business_counts


C:\Users\ASUS\AppData\Local\Temp\ipykernel_5756\3858994079.py:2: DtypeWarning: Columns (2,3,4,5,6,7,9,11,12,15,16,17,18,19) have mixed types. Specify dtype option on import or set low_memory=False.
  preprocessed_data = pd.read_csv(preprocessed_data_path)


(Empty DataFrame
 Columns: [angk, status_akhir, rencana_berangkat, rencana_kembali, desc, tujuan, realisasi_berangkat, realisasi_kembali, akumulasi_skor, nilaiPerilaku, durasi, hari_berangkat, hari_kembali, range_waktu, izin, tanggal_keluar, tanggal_masuk, lokasi, hari_keluar, hari_masuk, peluang_bisnis]
 Index: []
 
 [0 rows x 21 columns],
 Series([], Name: count, dtype: int64))

In [14]:
# Re-importing necessary library
import pandas as pd

# Reload the uploaded dataset and read the specific sheet
file_path = 'dataset.xlsx'
ik_data = pd.read_excel(file_path, sheet_name='DATA IK')

# Display the first few rows to understand its structure
ik_data.head()


,Unnamed: 0,Unnamed: 1,angk,status_akhir,rencana_berangkat,rencana_kembali,tujuan,realisasi_berangkat,realisasi_kembali
0,NaN,NaN,2019,Lulus,2022-08-08 15:00:00,2022-08-08 17:00:00,1. Keperluan IK: Memangkas/merapikan rambut_x0...,2022-08-08 15:06:05,2022-08-08 16:49:14
1,NaN,NaN,2019,Lulus,2022-08-10 12:20:00,2022-08-10 20:00:00,Keperluan IK : Melakukan pertemuan dengan Mitr...,2022-08-10 12:35:29,2022-08-10 19:38:44
2,NaN,NaN,2019,Lulus,2023-02-21 13:00:00,2023-02-21 17:00:00,Melakukan Market Research ke Homestay yang ada...,2023-02-21 13:45:21,2023-02-21 16:53:00
3,NaN,NaN,2019,Lulus,2023-04-13 13:00:00,2023-04-13 17:00:00,Mengunjungi keluarga yang sedang koma di RS Po...,2023-04-13 13:47:57,2023-04-13 16:46:45
4,NaN,NaN,2019,Lulus,2023-01-27 08:00:00,2023-01-27 15:00:00,Requirement ke BPBD untuk keperluan PA3,2023-01-27 08:56:45,2023-01-27 13:39:50


In [10]:
# Reload the dataset specifically targeting the "DATA IK" sheet again
ik_dataset = pd.read_excel(dataset_path, sheet_name='DATA IK')

# Display the first few rows to confirm successful loading
ik_dataset.head()


,Unnamed: 0,Unnamed: 1,angk,status_akhir,rencana_berangkat,rencana_kembali,tujuan,realisasi_berangkat,realisasi_kembali
0,NaN,NaN,2019,Lulus,2022-08-08 15:00:00,2022-08-08 17:00:00,1. Keperluan IK: Memangkas/merapikan rambut_x0...,2022-08-08 15:06:05,2022-08-08 16:49:14
1,NaN,NaN,2019,Lulus,2022-08-10 12:20:00,2022-08-10 20:00:00,Keperluan IK : Melakukan pertemuan dengan Mitr...,2022-08-10 12:35:29,2022-08-10 19:38:44
2,NaN,NaN,2019,Lulus,2023-02-21 13:00:00,2023-02-21 17:00:00,Melakukan Market Research ke Homestay yang ada...,2023-02-21 13:45:21,2023-02-21 16:53:00
3,NaN,NaN,2019,Lulus,2023-04-13 13:00:00,2023-04-13 17:00:00,Mengunjungi keluarga yang sedang koma di RS Po...,2023-04-13 13:47:57,2023-04-13 16:46:45
4,NaN,NaN,2019,Lulus,2023-01-27 08:00:00,2023-01-27 15:00:00,Requirement ke BPBD untuk keperluan PA3,2023-01-27 08:56:45,2023-01-27 13:39:50


In [22]:
# Define mapping for business opportunities based on keywords in 'tujuan'
business_opportunities = {
    "rambut": "Salon/barbershop",
    "pertemuan": "Sewa ruang pertemuan",
    "market research": "Konsultan riset",
    "keluarga": "Akomodasi pengunjung",
    "bpbd": "Layanan darurat",
}

# Function to categorize business opportunities
def map_business_opportunity(description):
    description = str(description).lower()
    for keyword, business in business_opportunities.items():
        if keyword in description:
            return business
    return "Lainnya"

# Apply the mapping to categorize opportunities
ik_data['peluang_bisnis'] = ik_data['tujuan'].apply(map_business_opportunity)

# Count the occurrences of each business opportunity
business_counts = ik_data['peluang_bisnis'].value_counts()

ik_data[['tujuan', 'peluang_bisnis']].head(), business_counts


(                                              tujuan        peluang_bisnis
 0  1. Keperluan IK: Memangkas/merapikan rambut_x0...      Salon/barbershop
 1  Keperluan IK : Melakukan pertemuan dengan Mitr...  Sewa ruang pertemuan
 2  Melakukan Market Research ke Homestay yang ada...       Konsultan riset
 3  Mengunjungi keluarga yang sedang koma di RS Po...  Akomodasi pengunjung
 4            Requirement ke BPBD untuk keperluan PA3       Layanan darurat,
 peluang_bisnis
 Lainnya                 1414
 Salon/barbershop          46
 Konsultan riset           46
 Akomodasi pengunjung      41
 Layanan darurat           28
 Sewa ruang pertemuan      15
 Name: count, dtype: int64)

In [23]:
# Filter data categorized as "Lainnya"
others_data = ik_data[ik_data['peluang_bisnis'] == "Lainnya"]

# Tampilkan deskripsi yang termasuk kategori "Lainnya"
print("Data dalam kategori 'Lainnya':")
print(others_data[['tujuan', 'peluang_bisnis']].head(20))  # Tampilkan 20 contoh pertama

# Analisis peluang baru: Mencari kata-kata yang sering muncul
from collections import Counter
import re

# Gabungkan semua teks dalam kategori "Lainnya"
all_text_others = " ".join(others_data['tujuan'].dropna().tolist()).lower()

# Bersihkan teks dan pecah menjadi kata-kata
words = re.findall(r'\b\w+\b', all_text_others)

# Hitung frekuensi setiap kata
word_counts = Counter(words)

# Tampilkan kata-kata yang paling sering muncul
print("\nKata-kata yang paling sering muncul dalam kategori 'Lainnya':")
print(word_counts.most_common(20))

# Filter data categorized as "Layanan darurat"
emergency_data = ik_data[ik_data['peluang_bisnis'] == "Layanan darurat"]

# Tampilkan data kategori "Layanan darurat"
print("\nData dalam kategori 'Layanan darurat':")
print(emergency_data[['tujuan', 'peluang_bisnis']].head(20))  # Tampilkan 20 contoh pertama


Data dalam kategori 'Lainnya':
                                               tujuan peluang_bisnis
7   Melakukan tahap analisis yaitu identifikasi ka...        Lainnya
8   Interview user terkait validasi MVP dari progr...        Lainnya
10  Requirement Gathering ke Dinas Perdagangan, Pe...        Lainnya
11  Requirement Gathering ke Dinas Perindustrian d...        Lainnya
13  Memperbaiki laptop karena diperlukan untuk per...        Lainnya
14  Untuk mengikuti ibadah natal youth yang disele...        Lainnya
16  Menjenguk orang tua (ibu) yang sedang sakit di...        Lainnya
17        Menjenguk orang tua (ibu) yang sedang sakit        Lainnya
18  Bertemu dengan partner (Hephata dan CBR-Toba) ...        Lainnya
20  Bertemu dengan partner (Hephata dan CBR-Toba) ...        Lainnya
21                                 Memperbaiki Laptop        Lainnya
24                                          Cek darah        Lainnya
25                                       Mengurus ATM        Lainnya
26 

In [38]:
# Define mapping for expanded business opportunities
business_opportunities = {
    "rambut": "Salon/barbershop",
    "pertemuan": "Sewa ruang pertemuan",
    "market research": "Konsultan riset",
    "keluarga": "Akomodasi pengunjung",
    "bpbd": "Transportasi atau penyewaan mobil",
    "laptop": "Toko dan layanan komputer",
    "komputer": "Toko dan layanan komputer",
    "gathering": "Transportasi atau penyewaan mobil",
    "acara": "Transportasi atau penyewaan mobil",
    "perjalanan jauh": "Transportasi atau penyewaan mobil",
    "control behel": "klinik gigi",
    "behel": "klinik gigi",
    "gigi": "klinik gigi",
    "mengikuti": "Transportasi atau penyewaan mobil",
"rumah sakit" : "Transportasi atau penyewaan mobil",
    "bertemu" : "Transportasi atau penyewaan mobil"
    
}

# Function to categorize business opportunities
def map_business_opportunity(description):
    description = str(description).lower()
    for keyword, business in business_opportunities.items():
        if keyword in description:
            return business
    return "Lainnya"

# Apply the mapping to categorize opportunities
ik_data['peluang_bisnis'] = ik_data['tujuan'].apply(map_business_opportunity)

# Count the occurrences of each business opportunity
business_counts = ik_data['peluang_bisnis'].value_counts()

# Filter data categorized as "Lainnya" for further analysis
others_data = ik_data[ik_data['peluang_bisnis'] == "Lainnya"]

# Display results
print("Peluang Bisnis yang Teridentifikasi:")
print(business_counts)

print("\nData dalam kategori 'Lainnya':")
print(others_data[['tujuan', 'peluang_bisnis']].head(20))  # Display first 20 rows for "Lainnya"


Peluang Bisnis yang Teridentifikasi:
peluang_bisnis
Lainnya                              1113
Transportasi atau penyewaan mobil     189
Toko dan layanan komputer              73
klinik gigi                            67
Salon/barbershop                       46
Konsultan riset                        46
Akomodasi pengunjung                   41
Sewa ruang pertemuan                   15
Name: count, dtype: int64

Data dalam kategori 'Lainnya':
                                               tujuan peluang_bisnis
7   Melakukan tahap analisis yaitu identifikasi ka...        Lainnya
8   Interview user terkait validasi MVP dari progr...        Lainnya
16  Menjenguk orang tua (ibu) yang sedang sakit di...        Lainnya
17        Menjenguk orang tua (ibu) yang sedang sakit        Lainnya
24                                          Cek darah        Lainnya
25                                       Mengurus ATM        Lainnya
26                       Penelitian ke Desa Sigapiton        Lainnya
30

In [40]:
import pandas as pd

# Load the uploaded Excel file
file_path = 'dataset.xlsx'

# Read the sheets "DATA IK" and "DATA IB"
data_ik = pd.read_excel(file_path, sheet_name='DATA IK')
data_ib = pd.read_excel(file_path, sheet_name='DATA IB')

# Display the first few rows of each sheet to understand their structure
data_ik.head(), data_ib.head()


(   Unnamed: 0  Unnamed: 1  angk status_akhir   rencana_berangkat  \
 0         NaN         NaN  2019        Lulus 2022-08-08 15:00:00   
 1         NaN         NaN  2019        Lulus 2022-08-10 12:20:00   
 2         NaN         NaN  2019        Lulus 2023-02-21 13:00:00   
 3         NaN         NaN  2019        Lulus 2023-04-13 13:00:00   
 4         NaN         NaN  2019        Lulus 2023-01-27 08:00:00   
 
       rencana_kembali                                             tujuan  \
 0 2022-08-08 17:00:00  1. Keperluan IK: Memangkas/merapikan rambut_x0...   
 1 2022-08-10 20:00:00  Keperluan IK : Melakukan pertemuan dengan Mitr...   
 2 2023-02-21 17:00:00  Melakukan Market Research ke Homestay yang ada...   
 3 2023-04-13 17:00:00  Mengunjungi keluarga yang sedang koma di RS Po...   
 4 2023-01-27 15:00:00            Requirement ke BPBD untuk keperluan PA3   
 
    realisasi_berangkat    realisasi_kembali  
 0  2022-08-08 15:06:05  2022-08-08 16:49:14  
 1  2022-08-10 12:35:29  2

In [42]:
# Define a mapping for business opportunities based on keywords
business_opportunities = {
    "rambut": "Salon/barbershop",
    "pertemuan": "Sewa ruang pertemuan",
    "market research": "Konsultan riset",
    "keluarga": "Akomodasi pengunjung",
    "bpbd": "Transportasi atau penyewaan mobil",
    "laptop": "Toko dan layanan komputer",
    "komputer": "Toko dan layanan komputer",
    "gathering": "Transportasi atau penyewaan mobil",
    "acara": "Transportasi atau penyewaan mobil",
    "perjalanan jauh": "Transportasi atau penyewaan mobil",
    "control behel": "klinik gigi",
    "behel": "klinik gigi",
    "gigi": "klinik gigi",
    "mengikuti": "Transportasi atau penyewaan mobil",
"rumah sakit" : "Transportasi atau penyewaan mobil",
    "bertemu" : "Transportasi atau penyewaan mobil"
}

# Function to categorize business opportunities based on description
def map_business_opportunity(description):
    description = str(description).lower()
    for keyword, business in business_opportunities.items():
        if keyword in description:
            return business
    return "Lainnya"

# Apply categorization for both datasets
data_ik['peluang_bisnis'] = data_ik['tujuan'].apply(map_business_opportunity)
data_ib['peluang_bisnis'] = data_ib['desc'].apply(map_business_opportunity)

# Count the number of opportunities per category for both datasets
ik_business_counts = data_ik['peluang_bisnis'].value_counts()
ib_business_counts = data_ib['peluang_bisnis'].value_counts()

# Display the categorized data and counts
data_ik[['tujuan', 'peluang_bisnis']].head(), ik_business_counts, data_ib[['desc', 'peluang_bisnis']].head(), ib_business_counts


(                                              tujuan  \
 0  1. Keperluan IK: Memangkas/merapikan rambut_x0...   
 1  Keperluan IK : Melakukan pertemuan dengan Mitr...   
 2  Melakukan Market Research ke Homestay yang ada...   
 3  Mengunjungi keluarga yang sedang koma di RS Po...   
 4            Requirement ke BPBD untuk keperluan PA3   
 
                       peluang_bisnis  
 0                   Salon/barbershop  
 1               Sewa ruang pertemuan  
 2                    Konsultan riset  
 3               Akomodasi pengunjung  
 4  Transportasi atau penyewaan mobil  ,
 peluang_bisnis
 Lainnya                              1113
 Transportasi atau penyewaan mobil     189
 Toko dan layanan komputer              73
 klinik gigi                            67
 Salon/barbershop                       46
 Konsultan riset                        46
 Akomodasi pengunjung                   41
 Sewa ruang pertemuan                   15
 Name: count, dtype: int64,
                        des